In [ ]:
import pandas as pd
import numpy as np
import time
import math

In [ ]:
tables_list = []
for i in range(2008, 2024, 1):
    time.sleep(0.5)
    print(i)
    if i in range(2010, 2016):
        tables_list.append(pd.read_html(f"https://en.wikipedia.org/wiki/{i}_FIFA_Ballon_d'Or"))
    elif i == 2020:
        continue
    elif i == 2016:
        tables_list.append(pd.read_html("https://en.wikipedia.org/wiki/2016_Ballon_d%27Or"))
    else:
        tables_list.append(pd.read_html(f"https://en.wikipedia.org/wiki/{i}_Ballon_d'Or"))

In [ ]:
tables_list[0][2]['year'] = [2008 for i in range(len(tables_list[0][2]))]
tables_list[0] = tables_list[0][2]

tables_list[1][2]['year'] = [2009 for i in range(len(tables_list[1][2]))]
tables_list[1] = tables_list[1][2]

tables_list[2] = pd.concat([tables_list[2][2], tables_list[2][3]], ignore_index=True)
tables_list[2]['year'] = [2010 for i in range(len(tables_list[2]))]

table = pd.concat([tables_list[3][2], tables_list[3][3]], ignore_index=True)
playas = []
for i in range(len(table)):
    if i in [0,1,2]:
        playas.append(table['Player[2]'][i])
    else:
        playas.append(table['Player[3]'][i])
table['player'] = playas
table['year'] = [2011 for i in range(len(table))]
tables_list[3] = table

n = 4
table = pd.concat([tables_list[n][2], tables_list[n][3]], ignore_index=True)
table['year'] = [2012 for i in range(len(table))]
tables_list[n] = table

n = 5
table = pd.concat([tables_list[n][2], tables_list[n][3]], ignore_index=True)
table['year'] = [2013 for i in range(len(table))]
tables_list[n] = table

n = 6
table = pd.concat([tables_list[n][2], tables_list[n][3]], ignore_index=True)
table['year'] = [2014 for i in range(len(table))]
tables_list[n] = table

n = 7
table = pd.concat([tables_list[n][2], tables_list[n][3]], ignore_index=True)
table['year'] = [2015 for i in range(len(table))]
tables_list[n] = table

tables_list[8][2]['year'] = [2016 for i in range(len(tables_list[8][2]))]
tables_list[8] = tables_list[8][2]

n=9
table = tables_list[n][2]
# table = pd.concat([tables_list[n][2], tables_list[n][3]], ignore_index=True)
table['year'] = [2017 for i in range(len(table))]
tables_list[n] = table

tables_list[10][2]['year'] = [2018 for i in range(len(tables_list[10][2]))]
tables_list[10] = tables_list[10][2]

tables_list[11] = pd.read_html("https://en.wikipedia.org/wiki/2019_Ballon_d'Or")
tables_list[11][2]['year'] = [2019 for i in range(len(tables_list[11][2]))]
tables_list[11] = tables_list[11][2]

tables_list[12][2]['year'] = [2021 for i in range(len(tables_list[12][2]))]
tables_list[12] = tables_list[12][2]

tables_list[13][2]['year'] = [2022 for i in range(len(tables_list[13][2]))]
tables_list[13] = tables_list[13][2]

# tables_list.append(pd.read_html("https://en.wikipedia.org/wiki/2023_Ballon_d'Or"))
tables_list[14][2]['year'] = [2023 for i in range(len(tables_list[14][2]))]
tables_list[14] = tables_list[14][2]

In [ ]:
big_table = pd.concat(tables_list, ignore_index=True)

In [ ]:
big_table.columns = [x.lower() for x in big_table.columns]

In [ ]:
print(list(big_table.columns.values))

In [ ]:
# renaming columns
# big_table.columns = ['rank', 'player0', 'nationality', 'club(s)', 'points', 'year', 'percent', 'player1', 'player2', 'player3', 'player4', 'votes', 'parser0', 'parser1', 'position', 'club']
big_table.columns = ['rank', 'player0', 'nationality', 'club(s)', 'points', 'year', 'percent', 'player1', 'player2', 'player3', 'player4', 'votes', 'position', 'club']
big_table.head()

In [ ]:
new_player_list = []
count_before = big_table.shape[0]
row_remove_idx = []
for idx, row in big_table.iterrows():
    # assigning players
    p0 = row['player0']
    p1 = row['player1']
    p2 = row['player2']
    p3 = row['player3']
    p4 = row['player4']

    non_nans = []
    nan_count = 0
    for i in [p0, p1, p2, p3, p4]:
        if not isinstance(i, str) and math.isnan(i):
            nan_count += 1
        else:
            non_nans.append(i)
    
    # only one non-nan value, so we can assume correctness
    if nan_count == 4 and len(non_nans) == 1:
        new_player_list.append(non_nans[0])
    
    # if multiple non-nan values but they are the same, we can assume correctness
    elif len(set(non_nans)) == 1:
        new_player_list.append(non_nans[0])
    
    # luckily there are never multiple players in same row
    elif len(set(non_nans)) >= 1:
        print('Multiple players in same row!?')

    # no player name in row, this occurs 5 times
    elif nan_count == 5:
        # uncommenting print and looking via manual inspection shows no useful information in rows, so we can skip them
        # print(row)
        row_remove_idx.append(idx)

count_after = len(new_player_list)
print(f'Lost {count_before-count_after} rows')

big_table.drop(row_remove_idx, inplace=True)

# adding new column
big_table['player'] = new_player_list

In [ ]:
# removing old columns and crazy columns
big_table = big_table[['year', 'player', 'rank', 'points', 'votes', 'percent', 'position', 'club', 'club(s)', 'nationality']].reset_index(drop=True)
big_table

In [ ]:
# many nans in points and votes especially makes me question their usability
print(big_table.shape[0])
print(big_table['points'].isna().sum())
print(big_table['votes'].isna().sum())

In [ ]:
# hopefully columns can just be mergeable and nans will disappear
print(big_table.shape[0])
print(big_table['club'].isna().sum())
print(big_table['club(s)'].isna().sum())

In [ ]:
one_nan_count = 0
both_nan_count = 0
for idx, row in big_table.iterrows():
    club = row['club']
    clubs = row['club(s)']

    if club == clubs:
        print('same')

    elif isinstance(clubs, str) and not isinstance(club, str) and math.isnan(club):
        # print(f'club is nan, club(s) is {clubs}')
        one_nan_count += 1

    elif isinstance(club, str) and not isinstance(clubs, str) and math.isnan(clubs):
        # print(f'club(s) is nan, club is {club}')
        one_nan_count += 1

    elif not isinstance(club, str) and math.isnan(club) and not isinstance(clubs, str) and math.isnan(clubs):
        # print('both are nan')
        both_nan_count += 1
    
print(one_nan_count)
print(both_nan_count)

# only one_nan_counts means we can merge the columns easily

In [ ]:
# removing old clubs and adding new club column with no nans
new_club = []
for idx, row in big_table.iterrows():
    club = row['club']
    clubs = row['club(s)']

    if isinstance(club, str):
        new_club.append(club)
    else:
        new_club.append(clubs)

big_table.drop(['club', 'club(s)'], axis=1, inplace=True)
big_table['club'] = new_club

In [ ]:
# replacing ranks that show as 1st etc. with just numbers
def replace_st_nd_rd_th(rank):
    if isinstance(rank, str):
        rank = rank.replace('st', '')
        rank = rank.replace('nd', '')
        rank = rank.replace('rd', '')
        rank = rank.replace('th', '')
    
    return int(rank)

big_table['rank'] = big_table['rank'].apply(replace_st_nd_rd_th)

In [ ]:
big_table

In [ ]:
print(big_table.shape[0])
print(big_table['year'].isna().sum())
print(big_table['player'].isna().sum())
print(big_table['rank'].isna().sum())
print(big_table['points'].isna().sum())
print(big_table['votes'].isna().sum())
print(big_table['position'].isna().sum())
print(big_table['percent'].isna().sum())
print(big_table['nationality'].isna().sum())
print(big_table['club'].isna().sum())

In [ ]:
# fill in nationalities by building dict of players and filling in after
# after googling there are actually some big players who changed nationality so it might be worth not getting rid of nans this way but getting them from another data source

nans_before = big_table['nationality'].isna().sum()

# build dict
player_nats = dict()
for idx, row in big_table.iterrows():
    player = row['player']
    nat = row['nationality']

    if player not in player_nats.keys():
        if isinstance(nat, str):
            player_nats[player] = nat

# make list corresponding to df
new_nats = []
for idx, row in big_table.iterrows():
    player = row['player']
    nat = row['nationality']

    if player in player_nats.keys():
        new_nats.append(player_nats[player])
    else:
        new_nats.append(np.nan)

# remove old column and add new
big_table.drop('nationality', axis=1, inplace=True)
big_table['nationality'] = new_nats

nans_after = big_table['nationality'].isna().sum()
print('nans before:', nans_before)
print('nans after:', nans_after)

In [ ]:
print(big_table.shape[0])
print(big_table['year'].isna().sum())
print(big_table['player'].isna().sum())
print(big_table['rank'].isna().sum())
print(big_table['points'].isna().sum())
print(big_table['votes'].isna().sum())
print(big_table['position'].isna().sum())
print(big_table['percent'].isna().sum())
print(big_table['nationality'].isna().sum())
print(big_table['club'].isna().sum())

### Checking for differences in csv and this data

In [ ]:
csv = pd.read_csv('../data/BallonDOr.csv')
csv

In [ ]:
print(csv.shape[0])
print(csv['Year'].isna().sum())
print(csv['Rank'].isna().sum())
print(csv['Player'].isna().sum())
print(csv['Club'].isna().sum())
print(csv['Nationality'].isna().sum())
print(csv['Points'].isna().sum())
print(csv['P1'].isna().sum())
print(csv['P2'].isna().sum())
print(csv['P3'].isna().sum())
print(csv['P4'].isna().sum())
print(csv['P5'].isna().sum())
print(csv['Votes'].isna().sum())
print(csv['RankPts'].isna().sum())
print(csv['Share'].isna().sum())
print(csv['Percent'].isna().sum())
print(csv['Voted'].isna().sum())

In [ ]:
big_table.year.unique()

In [ ]:
# checking for differences in data

# difference in number of records from 2008 to 2023
years_to_check = big_table.year.unique()
csv_years = csv[csv['Year'].isin(years_to_check)]
print(csv_years.shape[0])
print(big_table.shape[0])

# difference in rankings and player related data
for year in big_table.year.unique():
    print(year)
    big_table_ranks = big_table[big_table['year'] == year]['rank'].values
    csv_ranks = csv[csv['Year'] == 2008]['Rank'].values

    if len(big_table_ranks) > len(csv_ranks):
        if np.array_equal(big_table_ranks[:len(csv_ranks)], csv_ranks):
            print('More data in big table otherwise both the same!')
        else:
            print('More data in big table but mismatch in data!')
        
    elif len(big_table_ranks) < len(csv_ranks):
        if np.array_equal(big_table_ranks, csv_ranks[:len(big_table_ranks)]):
            print('More data in csv ranks otherwise both the same!')
        else:
            print('More data in csv ranks but mismatch in data!')
    
    else:
        if np.array_equal(big_table_ranks, csv_ranks):
            print('Same lengths and data matches!')
        else:
            print('Same lengths but data mimatch!')

    print(big_table_ranks)
    print(csv_ranks)

    # check if first 5 winners are the same
    print(big_table[big_table['year'] == year]['player'][:5])
    print(csv[csv['Year'] == year]['Player'][:5])

### Adding 2019-2021-2022-2023 years from big_table to csv

In [ ]:
big_table_relevant = big_table[big_table['year'].isin([2019, 2021, 2022, 2023])]

share = []
for year in [2019, 2021, 2022, 2023]:
    total_points = big_table_relevant[big_table_relevant['year']==year]['points'].sum()
    
    for idx, row in big_table_relevant[big_table_relevant['year']==year].iterrows():
        share.append(round(row['points']/total_points, 4))

big_table_relevant['share'] = share

In [ ]:
big_table_relevant

In [ ]:
# prepping for adding by removing unwanted columns and adding new ones
big_table_relevant.drop('position', axis=1, inplace=True)

big_table_relevant.columns = ['Year', 'Player', 'Rank', 'Points', 'Votes', 'Percent', 'Club', 'Nationality', 'Share']

big_table_relevant['P1'] = [np.nan] * big_table_relevant.shape[0]
big_table_relevant['P2'] = [np.nan] * big_table_relevant.shape[0]
big_table_relevant['P3'] = [np.nan] * big_table_relevant.shape[0]
big_table_relevant['P4'] = [np.nan] * big_table_relevant.shape[0]
big_table_relevant['P5'] = [np.nan] * big_table_relevant.shape[0]
big_table_relevant['RankPts'] = [np.nan] * big_table_relevant.shape[0]
big_table_relevant['Percent'] = [np.nan] * big_table_relevant.shape[0]
big_table_relevant['Voted'] = [np.nan] * big_table_relevant.shape[0]

In [ ]:
csv.columns

In [ ]:
big_table_relevant.columns

In [ ]:
df = pd.concat([csv, big_table_relevant])
df

In [ ]:
print(df.shape[0])
print(df['Year'].isna().sum())
print(df['Rank'].isna().sum())
print(df['Player'].isna().sum())
print(df['Club'].isna().sum())
print(df['Nationality'].isna().sum())
print(df['Points'].isna().sum())
print(df['P1'].isna().sum())
print(df['P2'].isna().sum())
print(df['P3'].isna().sum())
print(df['P4'].isna().sum())
print(df['P5'].isna().sum())
print(df['Votes'].isna().sum())
print(df['RankPts'].isna().sum())
print(df['Share'].isna().sum())
print(df['Percent'].isna().sum())
print(df['Voted'].isna().sum())

In [ ]:
df.to_csv('../data/BallonDOr_combined.csv', index=False)